<a href="https://colab.research.google.com/github/Alinehbg/EnsembleLearning/blob/Pauline/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from numpy import absolute
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.metrics import accuracy_score
import xgboost
from xgboost import XGBRegressor


In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd

# 0. Data Preprocessing

In [7]:
# load the clean data set
df_model = pd.read_csv('data_processed_v4.csv', index_col = 0)
df_model["price"] = df_model.index
df_model.index = range(len(df_model))

In [8]:
df_model.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,...,neighbourhood_1,neighbourhood_2,neighbourhood_3,neighbourhood_4,neighbourhood_5,neighbourhood_6,neighbourhood_7,neighbourhood_8,neighbourhood_9,neighbourhood_10
0,40.64749,-73.97237,0,1.0,9.0,0.21,6.0,365.0,0,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,40.75362,-73.98377,1,1.0,45.0,0.38,2.0,355.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40.80902,-73.94190,2,3.0,0.0,0.00,1.0,365.0,0,0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40.68514,-73.95976,3,1.0,270.0,4.64,1.0,194.0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40.79851,-73.94399,4,10.0,9.0,0.10,1.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
df_model['latitude'] = df_model['latitude'].astype('int')
df_model['longitude'] = df_model['longitude'].astype('int')
df_model['minimum_nights'] = df_model['minimum_nights'].astype('int')
df_model['number_of_reviews'] = df_model['number_of_reviews'].astype('int')
df_model['reviews_per_month'] = df_model['reviews_per_month'].astype('int')
df_model['calculated_host_listings_count'] = df_model['calculated_host_listings_count'].astype('int')
df_model['availability_365'] = df_model['availability_365'].astype('int')
df_model['neighbourhood_1'] = df_model['neighbourhood_1'].astype('int')
df_model['neighbourhood_2'] = df_model['neighbourhood_2'].astype('int')
df_model['neighbourhood_3'] = df_model['neighbourhood_3'].astype('int')
df_model['neighbourhood_4'] = df_model['neighbourhood_4'].astype('int')
df_model['neighbourhood_5'] = df_model['neighbourhood_5'].astype('int')
df_model['neighbourhood_6'] = df_model['neighbourhood_6'].astype('int')
df_model['neighbourhood_7'] = df_model['neighbourhood_7'].astype('int')
df_model['neighbourhood_8'] = df_model['neighbourhood_8'].astype('int')
df_model['neighbourhood_9'] = df_model['neighbourhood_9'].astype('int')
df_model['neighbourhood_10'] = df_model['neighbourhood_10'].astype('int')

In [10]:
# checking all the columns' type
with pd.option_context('display.max_rows', None):
    print(df_model.dtypes)

latitude                             int64
longitude                            int64
price                                int64
minimum_nights                       int64
number_of_reviews                    int64
reviews_per_month                    int64
calculated_host_listings_count       int64
availability_365                     int64
neighbourhood_group_Bronx            int64
neighbourhood_group_Brooklyn         int64
neighbourhood_group_Manhattan        int64
neighbourhood_group_Queens           int64
neighbourhood_group_Staten Island    int64
room_type_Entire home/apt            int64
room_type_Private room               int64
room_type_Shared room                int64
neighbourhood_1                      int64
neighbourhood_2                      int64
neighbourhood_3                      int64
neighbourhood_4                      int64
neighbourhood_5                      int64
neighbourhood_6                      int64
neighbourhood_7                      int64
neighbourho

# 1. Model

In [12]:
# Load the data into a pandas DataFrame
df = pd.read_csv("data_processed_v4.csv")
df.head()

,Unnamed: 0,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,...,neighbourhood_1,neighbourhood_2,neighbourhood_3,neighbourhood_4,neighbourhood_5,neighbourhood_6,neighbourhood_7,neighbourhood_8,neighbourhood_9,neighbourhood_10
0,0,40.64749,-73.97237,149.0,1.0,9.0,0.21,6.0,365.0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,40.75362,-73.98377,225.0,1.0,45.0,0.38,2.0,355.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,40.80902,-73.94190,150.0,3.0,0.0,0.00,1.0,365.0,0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,40.68514,-73.95976,89.0,1.0,270.0,4.64,1.0,194.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,40.79851,-73.94399,80.0,10.0,9.0,0.10,1.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Separate the features from the target variable
X = df.drop("price", axis=1)
y = df["price"]

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Define the XGBoost Regressor model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', seed=42)

*Note: `reg:squarederror` specifies that the model should use the mean squared error (MSE) as the objective function to minimize during training. This is appropriate for regression problems, where the goal is to predict a continuous numerical value, such as the price of a real estate property.*

In [16]:
# Define the hyperparameters to tune using cross-validation
parameters = {
              'colsample_bytree': [0.55, 0.6, 0.65],
              'learning_rate': [0.03, 0.04, 0.45],
              'max_bin': [0, 2, 256],
              'max_delta_step': [0, 0.5, 1]
              }

In [17]:
xgb_cv = GridSearchCV(
         estimator = xgb.XGBRegressor(objective='reg:squarederror',
                                      gamma = 0,
                                      max_depth = 7,
                                      max_leaves = 0,
                                      min_child_weight = 1.5,
                                      n_estimators = 140,
                                      reg_alpha = 5,
                                      reg_lambda = 0.75,
                                      scale_pos_weight = 0,
                                      subsample = 0.7,
                                      tree_method='gpu_hist',
                                      seed=42),
         param_grid = parameters,
         verbose = 2,
         cv = 10
)

In [18]:
# Fit the GridSearchCV object to the training data
xgb_cv.fit(X_train, y_train)

Fitting 10 folds for each of 81 candidates, totalling 810 fits
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.8s
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.2s
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.2s
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.1s
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.1s
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.1s
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.0s
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.1s
[CV] END colsample_bytree=0.55, learning_rate=0.03, max_bin=0, max_delta_step=0; total time=   0.1s
[CV] END colsample_bytree=0.55, learn

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
270 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.9/dist-packages/xgboost/sklearn.py", line 988, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "/usr/local/lib/python3.9/dist-packages/xgboost/sklearn.py"

GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=0, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=7, max_leaves=0,
                                    min_child_weight=1.5, missing=nan,
                                    monotone_constraints=None, n_estimators=140,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'colsample_bytree': [0.55, 0.6, 0.65],
                         'learning_rate': [0.03, 0.04, 0.45],
                         'max_bin': [0, 2, 256],
                         'max_delta_step': [0, 0.5, 1]},
             verbose=2)

In [19]:
# Print the best hyperparameters found by cross-validation
print("Best Hyperparameters: ", xgb_cv.best_params_)

Best Hyperparameters:  {'colsample_bytree': 0.55, 'learning_rate': 0.04, 'max_bin': 256, 'max_delta_step': 0}


In [20]:
xgb_cv.best_score_

0.19751962411770724

In [21]:
# Make predictions on the test data using the best model
y_pred = xgb_cv.predict(X_test)

In [22]:
# Evaluate the model's performance using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: ", mse)

Mean Squared Error:  29398.455683682794


In [23]:
import numpy as np

rmse = np.sqrt(mse)
print("Root Mean Squared Error: ", rmse)

Root Mean Squared Error:  171.45977861785192


In [25]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print("R-squared: ", r2)

R-squared:  0.23264883426010274
